# Your first CUDA code

In this lab, you will learn how to adapt a code such that it uses the GPU. 

Lab created by EPCC, The University of Edinburgh. Documentation and source code copyright The University of Edinburgh 2016. Lab style and template created by NVIDIA, see https://nvidia.qwiklab.com/.

---
First, please try and execute the below command. Give focus to the cell by clicking on it, and then either press the play button above or press your Enter key whilst holding down Shift.

In [ ]:
!echo "This command is running on host $HOSTNAME"

## Introduction

The exercise provides a template solution in either C or Fortran.

To edit a source file, we will use the notebook menus at the top right. You should still see
the directory structure in the original tab in your browser. Nagivate to the appropriate source file in `src_c` or `src_fortran`. Click on the filename to enter the editor. Use the file menu at the top right to save an updated version of the file.

The template source file is clearly marked with the sections to be edited, e.g.
<code>
/\* Part 1A: allocate device memory \*/
</code>
Please see below for instructions. Where necessary, you should refer to the CUDA C Programming Guide and Reference Manual documents available from
http://developer.nvidia.com/nvidia-gpu-computing-documentation
 


## 1) Copying Between Host and Device
This simple CUDA code has the purpose of negating an array of integers. We introduce the important concepts of device-memory management and kernel invocation. The final version should copy an array of integers from the host to device, multiply each element by −1 on the device, and then copy the array back to the host.

Choose the C or Fortran version by executing the corresponding cell below:


------
C:
--

In [ ]:
# set up a link to the C version of the templates
!rm -rf src; ln -s src_c src; echo "Using C version"

-------
Fortran:
--

In [ ]:
# set up a link to the Fortran version of the templates
!rm -rf src; ln -s src_fortran src; echo "Using Fortran version"

------
<b>C:</b>


Start from the intro.cu template.
<li> 1A) Allocate memory for the array on the device: use the existing pointer <code>d_a</code> and the variable
<code>sz</code> (which has already been assigned the size of the array in bytes).
<li> 1B) Copy the array <code>h_a</code> on the host to <code>d_a</code> on the device.
<li> 1C) Copy <code>d_a</code> on the device back to <code>h_out</code> on the host.
<li> 1D) Free <code>d_a</code>.

Execute the 2 cells below to compile and run the code (noting that the <code>arch</code> flag specifies the compute capability of the CUDA device).


--------
<b>Fortran:</b>


Start from the intro.cuf template.
<li> 1A) Allocate memory for the array on the device: use the existing pointer <code>d_a</code> and <code>ARRAY_SIZE</code> (which has already been assigned the size of the array in elements).
<li> 1B) Copy the array <code>h_a</code> on the host to <code>d_a</code> on the device, using an appropriate assignment operation.
<li> 1C) Copy <code>d_a</code> on the device back to <code>h_out</code> on the host, using another assignment operation.
<li> 1D) Deallocate <code>d_a</code>.

Execute the 2 cells below to compile and run the code (noting that the <code>Mcuda</code> flag specifies the compute capability of the CUDA device) 

----


In [ ]:
# Execute this cell to compile the code. Wait until "Complete" is printed in the output. 
!cd src; make clean; make; cd ..; echo "Complete"

In [ ]:
# Execute this cell to run the code. Wait until "Complete" is printed in the output.
!cd src; ./intro; cd ..; echo "Complete" 


------
The output (the contents of the h_out array) or any error messages will be printed. So far the code simply copies from h_a on the host to d_a on the device, then copies d_a back to h_out, so the‘ output should be the initial content of h_a — the numbers 0 to 255.

2) Launching Kernels
--

-----
<b>C:</b>

Now we will edit the intro.cu file to actually run a kernel on the GPU device.
<li> 2A) Configure and launch the kernel using a 1D grid and a single thread block (<code>NUM_BLOCKS</code>
and <code>THREADS_PER_BLOCK</code> are already defined for this case).
<li> 2B) Implement the actual kernel function to negate an array element as follows:
<code>
    int idx = threadIdx.x;
    d_a[idx] = -1 * d_a[idx];
</code>

<li> Compile and run the code by executing the above cells as before. This time the output should contain the result of negating each element of the input array. Because the array is initialised to the numbers 0 to 255, you should see the numbers 0 down to −255 printed.

This kernel works, but since it only uses one thread block, it will only be utilising one of the multiple SMs available on the GPU. Multiple thread blocks are needed to fully utilize the available resources.
<li> 2C) Implement the kernel again, this time allowing multiple thread blocks. It will be very similar to the previous kernel implementation except that the array index will be computed differently:
<code>
    int idx = threadIdx.x + (blockIdx.x * blockDim.x);
</code>    
Remember to also change the kernel invocation to invoke negate_multiblock this time. With this version you can change <code>NUM_BLOCKS</code> and <code>THREADS_PER_BLOCK</code> to have different values — so long as they still multiply to give the array size.

-----
<b>Fortran:</b>

Now we will edit the intro.cuf and array_negate.cuf files to actually run a kernel on the GPU device.
<li> 2A) Configure and launch the kernel using a 1D grid and a single thread block (<code>NUM_BLOCKS</code>
and <code>THREADS_PER_BLOCK</code> are already defined for this case).
<li> 2B) Implement the actual kernel function to negate an array element as follows:
<code>
integer :: idx

idx = threadidx%x
aa(idx) = -1*aa(idx)
</code>

<li> Compile and run the code by executing the above cells as before. This time the output should contain the result of negating each element of the input array. Because the array is initialised to the numbers 0 to 255, you should see the numbers 0 down to −255 printed.

This kernel works, but since it only uses one thread block, it will only be utilising one of the multiple SMs available on the GPU. Multiple thread blocks are needed to fully utilize the available resources.
<li> 2C) Implement the kernel again, this time allowing multiple thread blocks. It will be very similar to the previous kernel implementation except that the array index will be computed differently:
<code>
idx = threadidx%x + ((blockidx%x-1) * blockdim%x)
</code>    
Remember to also change the kernel invocation to invoke g_negate_multiblock this time. With this version you can change <code>NUM_BLOCKS</code> and <code>THREADS_PER_BLOCK</code> to have different values — so long as they still multiply to give the array size.


## 3) Handling any size array

At the moment we are insisting that the array size be an exact multiple of the block size. In general we should handle any size that will fit in GPU memory!

Let the total number of elements be `N ` and the block size be `B`. Recall that in integer division we discard the fractional part so we can write:
```
N = k * B + r
```
i.e. `N` can divided into `k` (an integer) number of blocks, plus a remainder, `r`. If `r` is zero, then we need `k` blocks, else we need `k + 1`. This can be expressed in a simple formula:
```
nBlocks = ((N-1) / B) + 1
```
Convince yourself this is correct!

* 3A) Update the kernel launch code to compute the number of blocks using this formula.

What will happen in the last block with the current kernel?

* 3B) Implement a condition in the kernel to protect against this.

Try changing `ARRAY_SIZE` to a non-multiple of 256 (e.g 500)


<a id="finished early"></a>
## Finally...

The <code>deviceQuery</code> utility queries the properties of the GPUs on the system. Execute the below cell and inspect the output.




In [ ]:
!/usr/local/home/course/deviceQuery

now add the text <code>export CUDA_VISIBLE_DEVICES=1,2;</code> to the line above (between the the ! and the <code>/usr</code>), and re-execute. Try again with <code>export CUDA_VISIBLE_DEVICES=1</code>. Try and work out what has happened.

<a id="post-lab"></a>
## Post-Lab

Finally, you may save your work from this lab before the end of the course.

From the file menu at the top right of the editor tab, click download.

<style>
p.hint_trigger{
  margin-bottom:7px;
  margin-top:-5px;
  background:#64E84D;
}
.toggle_container{
  margin-bottom:0px;
}
.toggle_container p{
  margin:2px;
}
.toggle_container{
  background:#f0f0f0;
  clear: both;
  font-size:100%;
}
</style>
<script>
$("p.hint_trigger").click(function(){
   $(this).toggleClass("active").next().slideToggle("normal");
});
   
$(".toggle_container").hide();
</script>